In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# define a list of features and label
x = [[1, 2], [3, 4], [5, 6], [7, 8]]
y = [[3], [7], [11], [15]]

# convert into pytorch tensor
x = torch.tensor(x).float()
y = torch.tensor(y).float()

# store the features and labels in device 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x = x.to(device)
y = y.to(device)

In [3]:
# define a dataset
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    
# dataset instance  
ds = MyDataset(x, y)

# fetch the batch_size from the dataset instance
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [4]:
# define a model
# mandatory to inherit from nn.module
class MyNeuralNet(nn.Module):
    def __init__(self):
        
        # take advantages of built_in functionnalities of nn.module 
        super().__init__()
        self.input_to_hidden_layer = nn.Parameter(torch.randn(2, 8))
        self.hidden_to_activation = nn.ReLU()
        self.hidden_to_output = nn.Parameter(torch.randn(8, 1))
        
    # forward pass
    def forward(self, x):
        x = x@self.input_to_hidden_layer
        x = self.hidden_to_activation(x)
        x = x@self.hidden_to_output
        return x

In [5]:
torch.manual_seed(10)
mynet = MyNeuralNet().to(device)

loss_func = nn.MSELoss()

# get the optimizer
from torch.optim import SGD
optimizer = SGD(mynet.parameters(), lr = 0.001)

loss_history = []

import time
start = time.time()
for epoch in range(50):
    for batches in dl:
        batch_x, batch_y = batches
        
        # flush the previous gradients
        optimizer.zero_grad()
        
        # compute the loss
        loss_value = loss_func(mynet(batch_x), batch_y)
        
        # backward propagation
        loss_value.backward()
        
        # upgrade the parameters
        optimizer.step()
        loss_history.append(loss_value.item())
end = time.time()
print('time : ', end - start)

time :  3.503028631210327


#
##### predictions

In [6]:
val_x = [[10,11]]

# convert into tensor of float dtype values and
# store it in device
val_x = torch.tensor(val_x).float().to(device)
mynet(val_x)

tensor([[20.9136]], device='cuda:0', grad_fn=<MmBackward0>)

In [7]:
for batches in dl:
    print(batches)

[tensor([[7., 8.],
        [3., 4.]], device='cuda:0'), tensor([[15.],
        [ 7.]], device='cuda:0')]
[tensor([[1., 2.],
        [5., 6.]], device='cuda:0'), tensor([[ 3.],
        [11.]], device='cuda:0')]
